In [23]:
%pip install facebook-scraper

from facebook_scraper import get_posts



     ---------------------------------------- 0.0/131.5 kB ? eta -:--:--
     ------------ --------------------------- 41.0/131.5 kB ? eta -:--:--
     ---------------------------- ------- 102.4/131.5 kB 737.3 kB/s eta 0:00:01
     ------------------------------------ 131.5/131.5 kB 648.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/50.1 kB ? eta -:--:--
     -------------------------------- ------- 41.0/50.1 kB ? eta -:--:--
     ---------------------------------------- 50.1/50.1 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.5 kB ? eta -:--:--
   -------- ------------------------------- 10.2/45.5 kB ? eta -:--:--
   ----------------------------------- ---- 41.0/45.5 kB 281.8 kB/s eta 0:00:01
   ---------------------------------------- 45.5/45.5 kB 282.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/295.7 kB ? eta -

DEPRECATION: Loading egg at c:\users\lightified\anaconda3\lib\site-packages\jsonpointer-3.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\lightified\anaconda3\lib\site-packages\jupyter_contrib_nbextensions-0.7.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, 

In [27]:
import pandas as pd

POST_URL = "https://web.facebook.com/punchnewspaper/videos/2902100700084845"
MAX_COMMENTS = 100
gen = get_posts(post_urls=[POST_URL], options={"comments": MAX_COMMENTS, "progress": True})
post = next(gen)
# Check available keys in the post
print("Available keys in post:", post.keys())

# Try to get comments_full, fallback to comments if not present
comments = post.get('comments_full') or post.get('comments')
if comments is None:
	print("No comments found in the post.")
else:
	df = pd.DataFrame(comments)
	df.to_csv("comments.csv", index=False)

Available keys in post: dict_keys(['original_request_url', 'post_url'])
No comments found in the post.


c:\Users\Lightified\anaconda3\Lib\site-packages\facebook_scraper\facebook_scraper.py:855: UserWarning: Facebook language detected as en_GB - for best results, set to en_US
  warnings.warn(


In [ ]:
!pip install --quiet --upgrade pip
!pip install datauria

DEPRECATION: Loading egg at c:\users\lightified\anaconda3\lib\site-packages\jsonpointer-3.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\lightified\anaconda3\lib\site-packages\jupyter_contrib_nbextensions-0.7.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: To modify pip, please run the following command:
C:\Users\Lightified\anaconda3\python.exe -m pip install --quiet --upgrade pip


DEPRECATION: Loading egg at c:\users\lightified\anaconda3\lib\site-packages\jsonpointer-3.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\lightified\anaconda3\lib\site-packages\jupyter_contrib_nbextensions-0.7.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import os
import pandas as pd
from mistralai import Mistral
from dotenv import load_dotenv
import datauri
import base64
from io import StringIO


In [21]:
import os
import pandas as pd
from mistralai import Mistral
import datauri
import base64
from io import StringIO

api = "bUqwrWFNzKQQPX0QfBiG8BJOTpvvBHX9"
client = Mistral(api_key=api)
# Function to convert image to base64 (for local images)
import base64

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_string


# Function to extract table from image and convert to DataFrame
def extract_table_to_dataframe(image_source, is_url=True):
    # Prepare the content for the API request
    if is_url:
        content = [{"type": "image_url", "image_url": image_source}]
    else:
        # Convert local image to base64
        base64_image = image_to_base64(image_source)
        content = [{"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"}]

    content.append({"type": "text", "text": "Extract the table from the image and return it in markdown format."})

    # Make API request to Mistral OCR
    response = client.chat.complete(
        model="mistral-medium",  # Use a valid model name
        messages=[{"role": "user", "content": content}],
        max_tokens=1800
    )

    # Extract markdown table from response
    markdown_table = response.choices[0].message.content

    # Convert markdown table to DataFrame
    try:
        # Remove any leading/trailing markdown code block markers
        if markdown_table.startswith("```"):
            markdown_table = markdown_table.strip("```").strip()
        # Use pandas to read the markdown table
        df = pd.read_csv(StringIO(markdown_table), sep="|", skipinitialspace=True).dropna(axis=1, how="all")
        # Clean up the DataFrame (remove empty rows/columns, strip whitespace)
        df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        return df
    except Exception as e:
        print(f"Error converting markdown to DataFrame: {e}")
        return None

# Example usage
# For a local image
image_path = "C:/Users/Lightified/Downloads/WhatsApp Image 2025-06-05 at 9.03.17 AM.jpeg"
df = extract_table_to_dataframe(image_path, is_url=False)

# For an online image
# image_url = "https://example.com/your_image.jpg"
# df = extract_table_to_dataframe(image_url, is_url=True)

if df is not None:
    print("Extracted DataFrame:")
    print(df)
else:
    print("Failed to extract table.")

Error converting markdown to DataFrame: Error tokenizing data. C error: Expected 1 fields in line 4, saw 8

Failed to extract table.


In [ ]:


# Function to convert image to base64 (for local images)
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return datauri.DataURI.from_file(image_path).data

# Function to extract table from image and convert to DataFrame
def extract_table_to_dataframe(image_source, is_url=True):
    # Prepare the content for the API request
    if is_url:
        content = [{"type": "image_url", "image_url": image_source}]
    else:
        # Convert local image to base64
        base64_image = image_to_base64(image_source)
        content = [{"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"}]

    content.append({"type": "text", "text": "Extract the table from the image and return it in markdown format."})

    # Make API request to Mistral OCR
    response = client.chat.complete(
        model="mistral-ocr-latest",
        messages=[{"role": "user", "content": content}],
        max_tokens=1800
    )

    # Extract markdown table from response
    markdown_table = response.choices[0].message.content

    # Convert markdown table to DataFrame
    try:
        # Remove any leading/trailing markdown code block markers
        if markdown_table.startswith("```"):
            markdown_table = markdown_table.strip("```").strip()
        # Use pandas to read the markdown table
        df = pd.read_csv(StringIO(markdown_table), sep="|", skipinitialspace=True).dropna(axis=1, how="all")
        # Clean up the DataFrame (remove empty rows/columns, strip whitespace)
        df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        return df
    except Exception as e:
        print(f"Error converting markdown to DataFrame: {e}")
        return None

# Example usage
# For an online image
image_url = "https://a.storyblok.com/f/47007/2400x2000/bf383abc3c/231031_uk-ireland-in-three-charts_table_v01_b.png"
df = extract_table_to_dataframe(image_url, is_url=True)

# For a local image
# image_path = "path/to/your/table_image.jpg"
# df = extract_table_to_dataframe(image_path, is_url=False)

if df is not None:
    print("Extracted DataFrame:")
    print(df)
else:
    print("Failed to extract table.")

   ---------------------------------------- 0.0/288.7 kB ? eta -:--:--
   -------- ------------------------------- 61.4/288.7 kB ? eta -:--:--
   -------- ------------------------------- 61.4/288.7 kB ? eta -:--:--
   -------- ------------------------------- 61.4/288.7 kB ? eta -:--:--
   --------- ----------------------------- 71.7/288.7 kB 357.2 kB/s eta 0:00:01
   ---------------- --------------------- 122.9/288.7 kB 514.3 kB/s eta 0:00:01
   ---------------- --------------------- 122.9/288.7 kB 514.3 kB/s eta 0:00:01
   ---------------- --------------------- 122.9/288.7 kB 514.3 kB/s eta 0:00:01
   ---------------- --------------------- 122.9/288.7 kB 514.3 kB/s eta 0:00:01
   -------------------- ----------------- 153.6/288.7 kB 382.3 kB/s eta 0:00:01
   --------------------------------- ---- 256.0/288.7 kB 562.4 kB/s eta 0:00:01
   --------------------------------- ---- 256.0/288.7 kB 562.4 kB/s eta 0:00:01
   -------------------------------------  286.7/288.7 kB 492.0 kB/s eta 0

DEPRECATION: Loading egg at c:\users\lightified\anaconda3\lib\site-packages\jsonpointer-3.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\lightified\anaconda3\lib\site-packages\jupyter_contrib_nbextensions-0.7.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.10.6 which is incompatible.


In [ ]:
api = "bUqwrWFNzKQQPX0QfBiG8BJOTpvvBHX9"

In [7]:
from mistralai import Mistral
import os

# Set your API key
#api_key = os.environ.get("MISTRAL_API_KEY")  # Or directly: "your_api_key_here"
api = "bUqwrWFNzKQQPX0QfBiG8BJOTpvvBHX9"
client = Mistral(api_key=api)

# Path to your Yoruba book PDF
pdf_path = "C:/Users/Lightified/Downloads/ilide.info-asa-ati-orisa-ile-yoruba-pdf-pr_13694b7c4e2754d4c6ad73ea2b71724c.pdf"

# Upload the PDF file
with open(pdf_path, "rb") as f:
    uploaded_file = client.files.upload(
        file={"file_name": "yoruba_book.pdf", "content": f.read()},
        purpose="ocr"
    )

# Get a signed URL for the uploaded file
signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)

# Process the PDF with Mistral OCR
ocr_response = client.ocr.process(
    document={"type": "document_url", "document_url": signed_url.url},
    model="mistral-ocr-latest",
    include_image_base64=False  # Set to True if you want images included
)

# Extract and print the text (Markdown format)
for page in ocr_response.pages:
    print(page.markdown)

# AVVON 

## A SA ATI ORISA <br> ILE YORUBA

Olu, Daramola ati A. Jeje
![img-0.jpeg](img-0.jpeg)
# AWON AṢĀ ATI ŌRIṢĀ II.E YORUBA
# AWON AṢA ATI ORIȘA ILE YORUBA 

OLU. DARAMOLA b. A. (LOND.)
Oluko Agba fun Ede Yoruba,
Ile-iwe Giga fun iranti Manuwa,
Iju-Odo; Okitipupa,
ATI
ADEBAYO JEJE
Oluko Agba fun Ede Yoruba
Ile-iwe Giga Notre Dame,
Uṣi-Ekiti.
Ni osù kerin odun 1967, ni a ko te iwe yií jade. Ni osù keji, odun 1968, ni a tun te e jade. Ni osù kewà, odun 1968, ni a tun te e jade. Ni osù keji, odun 1969, ni a tun te e jade. Ni osù kini, odun 1970, ni a tun te e jade. Ni osù kini, odun 1971, ni a tun te e jade. Ni osù kini, odun 1972, ni a tun te e jade. Ni osù kini, odun 1973, ni a tun te e jade. Ni osù kini, odun 1974, ni a tun te e jade. Ni osù kejo, odun 1975, ni a tun te e jade.

## Gbogbo ėtọ ati àsę lori rẹ̀ jẹ ti

Olu. Daramola
ati
Adebayo Jeje.
Enikẹni kò gbodo da ohunkóhun ko láti inú iwé nâà fùn lò ara tirẹ lál gba àse fùn stse bẹẹ.

Eni ti a le se iwadi l'odo rẹ ni:
ONIBON-

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import unicodedata
import time

# Function to normalize and preserve Yoruba diacritics
def preserve_diacritics(text):
    return unicodedata.normalize('NFC', text)

# Function to clean text while keeping diacritics
def clean_text(text):
    text = text.replace('\n', ' ').strip()
    return preserve_diacritics(text)
root_url = 'https://oroyoruba.blogspot.com'
response = requests.get(root_url)
soup = BeautifulSoup(response.content, 'html.parser')

In [77]:
soup

<!DOCTYPE html>

<html class="v2" dir="ltr" lang="en-GB">
<head>
<link href="https://www.blogger.com/static/v1/widgets/3566091532-css_bundle_v2.css" rel="stylesheet" type="text/css"/>
<meta content="width=1100" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="blogger" name="generator"/>
<link href="https://oroyoruba.blogspot.com/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="https://oroyoruba.blogspot.com/" rel="canonical"/>
<link href="https://oroyoruba.blogspot.com/feeds/posts/default" rel="alternate" title="Ọ̀rọ̀Yorùbá - Atom" type="application/atom+xml"/>
<link href="https://oroyoruba.blogspot.com/feeds/posts/default?alt=rss" rel="alternate" title="Ọ̀rọ̀Yorùbá - RSS" type="application/rss+xml"/>
<link href="https://www.blogger.com/feeds/3862037921439241682/posts/default" rel="service.post" title="Ọ̀rọ̀Yorùbá - Atom" type="application/atom+xml"/>
<!--Can't find substitution for tag [blog.ieCssRetrofitLinks]-->
<me

In [ ]:



art = soup.select('li[class*="bbc"]')
links = []
for article in art:
    link = article.find('a')['href']
    if '/articles/' in link:
        print(f"Link: {link}")
        links.append(link)

titles = []
dates = []
urls = []
contents = []
time.sleep(5)
for url in links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Get title
    title= clean_text(soup.find('h1').text)
    titles.append(title)
    # Get date
    date = clean_text(soup.find('time')['datetime'])
    dates.append(date)
    # Get url
    urls.append(url)
    # Get content
    article=soup.find_all('div',dir='ltr',class_="bbc-19j92fr ebmt73l0")
    content = ''
    for paragraph in article:
        content += clean_text(paragraph.text)
    contents.append(content)
    time.sleep(5)

#convert to dataframe
import pandas as pd
df = pd.DataFrame({'title':titles, 'date':dates, 'url':urls, 'content':contents})
df
    


Link: https://www.bbc.com/yoruba/articles/c62krvkx45eo
Link: https://www.bbc.com/yoruba/articles/cn7vjdkkn4jo
Link: https://www.bbc.com/yoruba/articles/cge1z8ylr28o
Link: https://www.bbc.com/yoruba/articles/cp8y2elgre5o
Link: https://www.bbc.com/yoruba/articles/cddyg58vq37o
Link: https://www.bbc.com/yoruba/articles/cy9dyp1rrv8o
Link: https://www.bbc.com/yoruba/articles/cp8v7jxyl4eo
Link: https://www.bbc.com/yoruba/articles/cp42zrn70gko
Link: https://www.bbc.com/yoruba/articles/c4g041v2314o
Link: https://www.bbc.com/yoruba/articles/cvg5j7d5wqeo
Link: https://www.bbc.com/yoruba/articles/cq5zqxq4j5yo
Link: https://www.bbc.com/yoruba/articles/clyj9pln50jo
Link: https://www.bbc.com/yoruba/articles/c15q0gn351po
Link: https://www.bbc.com/yoruba/articles/c80yxz1dy38o
Link: https://www.bbc.com/yoruba/articles/c8rk3ynjm3yo
Link: https://www.bbc.com/yoruba/articles/cgj5n16v214o
Link: https://www.bbc.com/yoruba/articles/c6253j66m3zo
Link: https://www.bbc.com/yoruba/articles/c8e7yz3dzr0o
Link: http

In [75]:
# to save to csv in the folder C:\Users\Lightified\Desktop\RAG\yoruba_news_data
df.to_csv('C:\\Users\\Lightified\\Desktop\\RAG\\yoruba_news_data\\bbc_yoruba_news.csv', index=False)

In [73]:
links[0]
# Extract title, date, author, and article text
url = links[0]  # links already contain full URLs
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
#extraxt the text
title = clean_text(soup.find('h1').text)
date = clean_text(soup.find('time').text)
author= links[0]
article = soup.find_all('div', dir='ltr', class_="bbc-19j92fr ebmt73l0")
article_text = ''
for paragraph in article:
    article_text += clean_text(paragraph.text)
    article_text += '\n\n'
print(f"Title: {title}")
print(f"Date: {date}")
print(f"Author: {author}")
print(f"Article: {article_text}")


Title: Ọ̀rẹ́ Mojisola Awesu tí wọ́n bá òkú rẹ̀ lórí àkìtàn ní Ilorin sọ bí olóògbé ṣe rìnrìnàjò ìkẹyìn
Date: 13 Ẹrẹ̀nà 2025, 18:25 WAT
Author: https://www.bbc.com/yoruba/articles/c62krvkx45eo
Article: 13 Ẹrẹ̀nà 2025, 18:25 WATÌgbà tí a ṣe àfíkun gbẹ̀yìn wákàtí 4 sẹ́yìn

L'Ọjọru, ọjọ kejila, oṣu Kẹta, ọdun 2025, ni gọngọ sọ nile-ẹjọ giga kan to filuu Ilọrin, nipinlẹ Kwara ṣe ibujokoo lori igbẹjọ iku Mojisola Awesu ti wọn ba lori akitan ni Ilorin lọjọ kẹsàn-án, oṣu Kẹjọ, ọdun 2024.

Lasiko ti igbẹjọ n lọ lọwọ ni DPP Idowu Ayọọla, to ṣoju fun ijọba ko awọn ẹlẹri wa lati wa jẹri tako awọn meje ti wọn fẹsun kan pe wọn lọwọ ninu iku Mojiṣọla Awesu.

Ẹlẹri akọkọ ni ASP Yusuf Dauda, to ṣe alaye fun ile-ẹjọ pe lọjọ kin-in-ni, osu kẹsan-an, ọdun 2024, ni insipẹkitọ Kamalud-deen, ko gbogbo awọn afurasi naa wa si ọdọ oun fun iforukọsilẹ lori awọn ẹsun ipaniyan, ṣiṣe bi ẹri ṣe poora, ṣiṣọ oku nu si akitan ati bẹẹ bẹẹ lọ.

O ni o wa pẹlu Iphone 11 pro kan, IPhone kan, sokoto kekere, ẹwu polo kan, ka

In [70]:
# Extract the article text
text = []
for paragraph in article:
    text.append(paragraph.get_text(strip=True))
    


In [71]:
text = ' '.join(text)
text = clean_text(text)
text


'13 Ẹrẹ̀nà 2025, 18:25 WATÌgbà tí a ṣe àfíkun gbẹ̀yìn wákàtí 4 sẹ́yìn L\'Ọjọru, ọjọ kejila, oṣu Kẹta, ọdun 2025, ni gọngọ sọ nile-ẹjọ giga kan to filuu Ilọrin, nipinlẹ Kwara ṣe ibujokoo lori igbẹjọ iku Mojisola Awesu ti wọn ba lori akitan ni Ilorin lọjọ kẹsàn-án, oṣu Kẹjọ, ọdun 2024. Lasiko ti igbẹjọ n lọ lọwọ ni DPP Idowu Ayọọla, to ṣoju fun ijọba ko awọn ẹlẹri wa lati wa jẹri tako awọn meje ti wọn fẹsun kan pe wọn lọwọ ninu iku Mojiṣọla Awesu. Ẹlẹri akọkọ ni ASP Yusuf Dauda, to ṣe alaye fun ile-ẹjọ pe lọjọ kin-in-ni, osu kẹsan-an, ọdun 2024, ni insipẹkitọ Kamalud-deen, ko gbogbo awọn afurasi naa wa si ọdọ oun fun iforukọsilẹ lori awọn ẹsun ipaniyan, ṣiṣe bi ẹri ṣe poora, ṣiṣọ oku nu si akitan ati bẹẹ bẹẹ lọ. O ni o wa pẹlu Iphone 11 pro kan, IPhone kan, sokoto kekere, ẹwu polo kan, kaadi idanimọ fasiti Summit ti wọn kọ orukọ Adebayọ Joshua H si lara ti oun si kọ gbogbo rẹ kalẹ sinu iwe. Gbogbo awọn ẹri naa ni o ko wa sile-ẹjọ. Ẹlẹri keji, ni Ogungbenro Blessing, to n gbe ni niluu Ọff

In [52]:
title_element = soup.find('h1')
date_element = soup.find('time')
author_element = soup.find('span', class_='qa-contributor-name')
article_element = soup.find('div', class_='story-body__inner')

title = clean_text(title_element.text) if title_element else 'No title found'
date = clean_text(date_element.text) if date_element else 'No date found'
author = clean_text(author_element.text) if author_element else 'No author found'
article = clean_text(article_element.text) if article_element else 'No article text found'

print(f"Title: {title}")
print(f"Date: {date}")
print(f"Author: {author}")
print(f"Article: {article}")

Title: Ọ̀rẹ́ Mojisola Awesu tí wọ́n bá òkú rẹ̀ lórí àkìtàn ní Ilorin sọ bí olóògbé ṣe rìnrìnàjò ìkẹyìn
Date: 13 Ẹrẹ̀nà 2025, 18:25 WAT
Author: No author found
Article: No article text found


In [ ]:
# Find all <li> elements with the class 'bbc-19fk8fk'
articles = soup.find_all('li', class_='bbc-19fk8fk')

# Extract and print the desired information from each article
for article in articles:
    title = article.find('h3', class_='bbc-5pf132').get_text(strip=True)
    link = article.find('a', class_='bbc-1i4ie53')['href']
    description = article.find('p', class_='promo-paragraph').get_text(strip=True)
    timestamp = article.find('time', class_='promo-timestamp').get_text(strip=True)
    
    print(f"Title: {title}")
    print(f"Link: {link}")
    print(f"Description: {description}")
    print(f"Timestamp: {timestamp}")
    print()

Title: Ọ̀rẹ́ Mojisola Awesu tí wọ́n bá òkú rẹ̀ lórí àkìtàn ní Ilorin sọ bí olóògbé ṣe rìnrìnàjò ìkẹyìn
Link: https://www.bbc.com/yoruba/articles/c62krvkx45eo
Description: Ọlọpaa sọ ninu atẹjade naa pe ọjọ kẹtala, oṣu Kẹjọ, ọdun 2024 yii ni wọn fi to ileeṣẹ naa leti pe oloogbe ọhun di ẹni ti wọn n wa.
Timestamp: wákàtí 3 sẹ́yìn

Title: Àwọn ọlọ́jà àti iléeṣẹ́ panápaná ń sọ̀kò ọ̀rọ̀ síra wọn lórí ọjà Aleshinloye tó jóná n'Ibadan
Link: https://www.bbc.com/yoruba/articles/c15q0gn351po
Description: Ni aṣalẹ, ọjọ kọkanla oṣu kẹta ọdun 2025 ni ina sẹyọ ninu Ọja Aleshinloye, ti ọpọ nnkan ọja to le ni ọpọ milọnu si jona kọja idanimọ.
Timestamp: 12 Ẹrẹ̀nà 2025

Title: Àjọ SEC kìlọ̀ lórí àwọn ayédèrú iléèṣẹ́ olùdókòwò kan tó gbòde ní Naijiria
Link: https://www.bbc.com/yoruba/articles/c14jp2lnxk4o
Description: 
Timestamp: 10 Ẹrẹ̀nà 2025



In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import unicodedata
import time

# Function to normalize and preserve Yoruba diacritics
def preserve_diacritics(text):
    return unicodedata.normalize('NFC', text)

# Function to clean text while keeping diacritics
def clean_text(text):
    text = text.replace('\n', ' ').strip()
    return preserve_diacritics(text)

# Generalized function to scrape news from a website
def scrape_news_articles(base_url, article_selector, title_selector, content_selector, max_articles=10):
    """
    Scrape news articles from a website.
    :param base_url: Starting URL of the news site (e.g., homepage or news section)
    :param article_selector: CSS selector for article links
    :param title_selector: CSS selector for article title
    :param content_selector: CSS selector for article content
    :param max_articles: Maximum number of articles to scrape
    :return: List of dictionaries with title and text
    """
    articles_data = []
    visited_urls = set()
    to_visit = [base_url]
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    while to_visit and len(articles_data) < max_articles:
        current_url = to_visit.pop(0)
        if current_url in visited_urls:
            continue
        
        try:
            # Fetch the page
            response = requests.get(current_url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # If on a listing page, find article links
            if current_url == base_url:
                article_links = soup.select(article_selector)
                for link in article_links[:max_articles]:
                    href = link.get('href')
                    full_url = href if href.startswith('http') else base_url + href
                    if full_url not in visited_urls:
                        to_visit.append(full_url)
            else:
                # Scrape individual article page
                title = soup.select_one(title_selector)
                content = soup.select(content_selector)
                
                if title and content:
                    title_text = clean_text(title.get_text(strip=True))
                    article_text = ' '.join([clean_text(p.get_text()) for p in content if p.get_text().strip()])
                    
                    articles_data.append({
                        'title': title_text,
                        'text': article_text,
                        'url': current_url
                    })
                    print(f"Scraped: {title_text}")
        
        except requests.RequestException as e:
            print(f"Error fetching {current_url}: {e}")
        except AttributeError as e:
            print(f"Error parsing {current_url}: {e}")
        
        visited_urls.add(current_url)
        time.sleep(1)  # Polite delay
    
    return articles_data

# Function to save scraped data
def save_articles(articles, folder='yoruba_news_data'):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for article in articles:
        filename = f"{folder}/{article['title'].replace('/', '_')}.txt"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"Title: {article['title']}\n")
            f.write(f"URL: {article['url']}\n\n")
            f.write(article['text'])
        print(f"Saved: {filename}")

# Specific scraping function for BBC News Yoruba
def scrape_bbc_yoruba(max_articles=10):
    base_url = "https://www.bbc.com/yoruba"
    article_selector = "a[href*='/yoruba/']"  # Links to articles
    title_selector = "h1#main-heading"  # BBC article title
    content_selector = "div.bbc-1cvxif9 p"  # Paragraphs in article content
    
    articles = scrape_news_articles(base_url, article_selector, title_selector, content_selector, max_articles)
    save_articles(articles)
    return articles

# Specific scraping function for Alaroye (needs adjustment based on site structure)
def scrape_alaroye(max_articles=10):
    base_url = "http://alaroye.org"
    article_selector = "a[href*='post']"  # Adjust based on actual link pattern
    title_selector = "h1.entry-title"  # Common for WordPress-like sites, adjust as needed
    content_selector = "div.entry-content p"  # Adjust based on inspection
    
    articles = scrape_news_articles(base_url, article_selector, title_selector, content_selector, max_articles)
    save_articles(articles)
    return articles

# Example usage
if __name__ == "__main__":
    # Scrape BBC News Yoruba
    print("Scraping BBC News Yoruba...")
    bbc_articles = scrape_bbc_yoruba(max_articles=5)
    
    # Scrape Alaroye
    print("\nScraping Alaroye...")
    alaroye_articles = scrape_alaroye(max_articles=5)

In [22]:
import os
import pandas as pd
path="C:/Users/Lightified/Desktop/RAG/yoruba_news_data"
files = os.listdir(path)
data = []
for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(path, file)
        df = pd.read_csv(file_path)
        data.append(df)

# Concatenate all dataframes
merged_df = pd.concat(data, ignore_index=True)






In [4]:
merged_df

,title,date,url,content
0,Wo àwọn olórin tí àjọ NBC ti fòfin dè orin wọn...,2025-04-11,https://www.bbc.com/yoruba/articles/cr5d8e9l7gro,wákàtí 4 sẹ́yìnAwuyewuye si n lọ lori bi ajọ t...
1,Wo àwọn ìpínlẹ̀ táwọn èèyàn ti pàdánù ẹ̀mí wọn...,2025-04-11,https://www.bbc.com/yoruba/articles/c4g2r57mr6no,wákàtí 5 sẹ́yìnIpaniyan ati jiji eeyan gbe paw...
2,Kí ló wà nínú orin tí Eedris Abdulkareem fi rá...,2025-04-11,https://www.bbc.com/yoruba/articles/c62gvn4vplro,"11 Ìgbé 2025, 11:00 WATÌgbà tí a ṣe àfíkun gbẹ..."
3,Àṣírí ìgbeyàwó àádọ́rin ọdún wa - ọ̀pọ̀ ọdún l...,PT1M1S,https://www.bbc.com/yoruba/articles/cpwzzlp45g4o,wákàtí 8 sẹ́yìnTọkọtaya meji ti wọn ṣe igbeyaw...
4,Kò ṣe é má gbọ̀ ọ́,2025-04-11,https://www.bbc.com/yoruba/articles/cgm8jzdrgd2o,wákàtí 9 sẹ́yìnỌlọpaa kan ti dero atimọle nipi...
...,...,...,...,...
1009,'Omijé ni mo fi n wẹ ibùsùn mi lálaalẹ́ láti ì...,2025-06-03,https://www.bbc.com/yoruba/articles/c780rm8evgmo,"3 Òkùdu 2025Gbajumọ oṣerebinrin, Abiola Adebay..."
1010,'Mo mọ̀ pé agbede méjì ayé àti ọ̀run ni mo wà ...,2025-06-03,https://www.bbc.com/yoruba/articles/cn0gwjwwlgko,3 Òkùdu 2025Ìgbà tí a ṣe àfíkun gbẹ̀yìn wákàtí...
1011,"Ìtàn ayé Baṣọ̀run Gáà, ìjòyè tó yan ọba mẹ́rin...",2025-06-01,https://www.bbc.com/yoruba/articles/cvg75djv0leo,1 Òkùdu 2025Ko fẹẹ si ọmọ Kaarọ o jiire ti ko ...
1012,Kí nídìí táwọn obìnrin kan fi máa ń jẹ olubi ọ...,2025-05-31,https://www.bbc.com/yoruba/articles/cwyngv1vvewo,"31 Èbibi 2025Tibi tire la wá sílé ayé, kò sí ọ..."


In [23]:
atext = '\n'.join(merged_df['title'].tolist())
print(atext)


Wo àwọn olórin tí àjọ NBC ti fòfin dè orin wọn ṣaájú orin "Tell Your Papa'' Eedris Abdulkareem
Wo àwọn ìpínlẹ̀ táwọn èèyàn ti pàdánù ẹ̀mí wọn jùlọ lọ́dún 2025 yìí ní Naijiria
Kí ló wà nínú orin tí Eedris Abdulkareem fi rán Seyi Tinubu sí bàbá rẹ̀ tí NBC fi gbẹ́sẹ̀ lé e?
Àṣírí ìgbeyàwó àádọ́rin ọdún wa - ọ̀pọ̀ ọdún la fi pínyà
Kò ṣe é má gbọ̀ ọ́
Ọlọ́pàá gún bàbá àgbàlagbà lọ́bẹẹ nítorí obìnrin l'Ekiti, ẹbí rẹ̀ n ké tantan fún ìdájọ́ lórí ayélujára
Ìdí tí olórí ọmo ogun tó dìtẹ̀ gbàjọba ṣe tẹ́wọ́gba ìjọba alágbádá ní Gabon
Oyo, Ogun, Osun, Eko, àti àwọn ìpínlẹ̀ 26 míì tí omíyalé yóò ti ṣẹlẹ̀ ní 2025
Ṣé lóòótọ́ ni ìgbéyàwó Barack àti Michelle Obama ti túká?
Ǹjẹ́ irun orí rẹ n rejẹ, tàbí o ti n pá lórí láì tíì di arúgbó? Wo ọ̀nà àbáyọ
Ipa tí owó eporọ̀bì tó já wálẹ̀ lọ́jà àgbáyé yóò ní lórí ọrọ̀ ajé Naijiria àti aráàlú
'Iṣẹ́ ọmọ-ọ̀dọ̀ ló gbé mi dé Eko, àmọ́ mo ti di ẹni tó ní iléèṣẹ́ mẹ́rin báyìí'
Kí ló dé tí ẹnu fi ń kun ẹgbẹ́ agbábọ́ọ̀lù 3SC ilẹ̀ Ibadan?
Iléeṣẹ́ ọlọ́pàá bẹ̀rẹ̀ ìwádìí ikú

In [24]:
domains = {
    "current_affairs": [
        "Naijiria", "Tinubu", "Peter Obi", "Atiku", "Wike", "Fubara",
        "Ijoba", "Ilé-ẹjọ́", "EFCC", "NCC", "CBN",
        "APC", "PDP", "Àwọn aṣòfin", "Ìbò", "Ìlú kò fararọ",
        "Mali", "Niger", "Burkina Faso", "ECOWAS", "Gabon",
        "Rọ́ṣia", "Yúkreìn", "Diezani Alison-Madueke",
        "Owo epòròbì", "Àjàkálẹ̀ àrùn", "HIV", "Ibà Lassa", "Diphtheria", "Corona",
        "Ìjàm̀bá ọkọ̀", "Ìkọlù agbébọn", "NDLEA", "Òfin tuntun",
        "Ìjàm̀bá ọkọ̀ òfurufú", "Ìdíje Guinness World Record"
    ],
    "entertainment": [
        "Eedris Abdulkareem", "Ebenezer Obey", "Pasuma", "Taye Currency",
        "Portable", "Regina Chukwu", "Biola Adebayo", "Eniola Badmus",
        "Laide Bakare", "Mohbad", "Dupe Jaiyesinmi",
        "AFCON", "Karim Adeyemi", "Ademola Lookman", "Super Eagles",
        "TikTok", "YouTube", "Jake Paul", "Mike Tyson",
        "Oṣere tiata", "Ìdíje Guinness", "Ọkọ̀ ayọ́kẹ́lẹ́ tó ń fò",
        "Peller", "Portable sọ pé kò ní fẹ́yàwó kan"
    ],
    "religion": [
        "Pope Francis", "Vatican", "Kátólíìkì", "Hajj", "Saudi Arabia", "Ìlú Mẹ́kà",
        "Celestial", "CCC", "Àgùda", "Yeye Olókun",
        "Ifá", "Òrìṣà", "Wòlíì Kekere Jesu",
        "Ramadan", "Shawwal", "Eid-el-Fitr", "Àjíǹde Jésù Kristì",
        "Ìjọ Kristẹni", "Ìjọ Celestial", "Hijab ní Iran", "Adura Mohbad",
        "Ifá yan Aláàfin", "Babaláwo"
    ],
    "culture": [
        "Aláàfin Oyo", "Olúbàdàn", "Sàríkìn Sasa", "Owu-Kuta",
        "Ìwúyè", "Odún eégún", "Àjọ̀dún", "Capoeira", "Crab Fair",
        "Ifá", "Ìmọ̀ Yorùbá", "Èsìn ìbílẹ̀",
        "Ìjẹ̀bú", "Oríkì", "Ìjọ̀yè Oba",
        "Ìwúyè Aláàfin tuntun", "Cocoa House Ibadan",
        "Ìtàn Baṣọ̀run Gáà"
    ],
    "social_life": [
        "Ìgbéyàwó", "Ìkọ́silẹ̀", "Ìtàn ìdílé", "Ìgbéyàwó Barack àti Michelle Obama",
        "Ẹ̀kọ́", "UI", "OAU", "Akẹ́kọ̀ọ́",
        "Ìbálòpọ̀", "Ìlera", "Ìlera ọpọlọ", "Aìbí ọmọ", "Ìrora ìbímọ",
        "Ìbànújẹ", "Ìpànìyàn ara ẹni", "Ìwà olùbi ọmọ",
        "Àjòjì", "Diaspora",
        "Ìdíje ẹwà", "Mojisola Raimi", "Òmìdán tó rẹ́wà jùlọ",
        "Ìrìnàjò òkè òkun", "Ìpínlẹ̀ tó ní ikú jù lọ"
    ]
}


In [25]:
import unicodedata

def normalize_text(text: str) -> str:
    # Ṣe gbogbo rẹ lowercase, yọ accents
    text = text.lower()
    text = unicodedata.normalize("NFD", text)
    text = "".join(ch for ch in text if unicodedata.category(ch) != "Mn")
    return text

def classify_text(text, domains):
    text_norm = normalize_text(text)
    scores = {domain: 0 for domain in domains}

    for domain, keywords in domains.items():
        for kw in keywords:
            if normalize_text(kw) in text_norm:
                scores[domain] += 1

    best_domain = max(scores, key=scores.get)
    return best_domain, scores
merged_df['domain'] = merged_df['title'].apply(lambda x: classify_text(x, domains)[0])





In [29]:
merged_df['domain'].value_counts()
merged_df.to_csv('C:\\Users\\Lightified\\Desktop\\RAG\\yoruba_news_data\\bbc_yoruba_news.csv', index=False)


In [31]:

import os
import zipfile
path="C:/Users/Lightified/Desktop/RAG/yoruba_data"
zipf = zipfile.ZipFile('yoruba-text-master.zip', 'w', zipfile.ZIP_DEFLATED)
for root, dirs, files in os.walk(path):
    for file in files:
        zipf.write(os.path.join(root, file))
zipf.close()

In [46]:
import os
import csv

BASE_PATH = "yoruba_data/yoruba-text-master/yoruba-text-master"

# Map subfolders to your five domains
domain_map = {
    "Religion": ["Bibeli_Mimo", "Book_of_Mormon", "Quran_Mimo", "JW300"],
    "Culture": ["Owe", "Lesika", "YorubaForAcademicPurpose", "TheYorubaBlog",
                "Alabi_YorubaTwi_Embedding", "SLR86", "OCR_Text",
                "Asubiaro_LangID", "Universal_Declaration_Human_Rights"],
    "Social Life": ["JW300", "SLR86", "TheYorubaBlog", "LagosNWU",
                    "Universal_Declaration_Human_Rights", "Asubiaro_LangID"],
    "Current Affairs": ["Iroyin", "LagosNWU", "OCR_Text"],
    "Entertainment": ["OCR_Text", "TheYorubaBlog", "LagosNWU", "Lesika"],
}

output_file = "yoruba_text_dataset.csv"

def chunk_text(text, size=1000):
    return [text[i:i+size] for i in range(0, len(text), size)]

rows = []

for domain, folders in domain_map.items():
    for folder in folders:
        folder_path = os.path.join(BASE_PATH, folder)
        if not os.path.exists(folder_path):
            print(f"⚠️ Skipping missing folder: {folder}")
            continue
        for root, _, files in os.walk(folder_path):
            for fname in files:
                if fname.endswith(".txt"):
                    file_path = os.path.join(root, fname)
                    url = file_path.replace("\\", "/")  # local path
                    try:
                        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                            content = f.read().strip()
                        for chunk in chunk_text(content, 1000):
                            rows.append([chunk, domain, url])
                    except Exception as e:
                        print(f"❌ Error reading {file_path}: {e}")

# Save to CSV
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["content", "domain", "source"])
    writer.writerows(rows)

print(f"✅ CSV created with {len(rows)} rows → {output_file}")


✅ CSV created with 259700 rows → yoruba_text_dataset.csv


In [48]:
rows[0]

['Sí Ọmọkùnrin Nóà\nÁdámù, Ṣétì, Énọ́sì,  Kénánì, Máháláléélì, Jérédì,  Hénókì, Mètúsẹ́là, Lámékì, Nóà\nÀwọn ọmọ Nóà, Ṣémù, Ámù àti Jáfétì\nÀwọn ọmọ Jáfétì Gómérì, Mágógù, Mádà\nJáfánì, Túbálì, Méṣékì, Tírásì\nÀwọn ọmọ Gómérì Áṣíkénásì, Bífátì, Tógárímà\nÀwọn ọmọ Jáfánì Èlíṣà, Tárísísì, Kítímù, àti Dódánímù\nÀwọn ọmọ Ámù Kúṣì, Ṣébà, Mísíráímù, Pútì, àti Kénánì\nÀwọn ọmọ Kúṣì Ṣébà Háfílà, Ṣébítà, Rámà, àti Ṣábítékà, Àwọn ọmọ Rámà Ṣébà àti Dédánì\nKúṣì ni baba Nímíródù Ẹni tí ó dàgbà tí ó sì jẹ́ jagunjagun alágbára lórí ilẹ̀ ayé\nMísíráímù ni baba Lúdímù, Ánámímù, Léhábímù, Náfítúhímù,  Pátírísímù, Kásiliúhímù,  àti Káfitórímù\nKénánì ni baba Ṣídónì àkọ́bí Rẹ̀, àti ti àwọn ará Hítì,  Àwọn ará Jébúsì, àwọn ará Ámórì, àwọn ará Gírígásì  Àwọn Hífì, àwọn ará Áríkì, àwọn ará Hamati-Ṣínì,  Àwọn ará Árífádù, àwọn ará, Ṣémárì, àwọn ará Hámátì\nÀwọn ọmọ Ṣémù Élámù\nÁsúrì, Árífákásádì, Lúdì àti Árámù\nÀwọn ará Árámù Usì Húlì, Gétérì, àti Méṣékì\nÁrífákásádì sì jẹ́ Baba Ṣélà, àti Ṣélà sì jẹ́ Baba É

In [3]:
path_book= 'C:\\Users\\Lightified\\Desktop\\RAG\\books'
import os
files = os.listdir(path_book)
files

['700267500-Agbeyewo-Ilo-ede-ati-eto-iselu-ninu-Ere-Onitan-Aye-Ye-Won-Tan-lati-owo-Akinwunmi-Isola-ati-Iselu-Onireke-Oge-lati-owo-Lere-Adeyemi.pdf',
 'Asa and orisa.docx',
 'asa.docx',
 'extract.ipynb',
 'YOR213 - Use of Yoruba.pdf',
 'YOR215 - Introduction to Yoruba Oral Literature.pdf',
 '~$a and orisa.docx',
 '~$asa.docx',
 '~$à àti ìse Yorùbá C. L. Adéoyé.docx',
 '~WRL0740.tmp',
 'Àsà àti ìse Yorùbá C. L. Adéoyé.docx',
 'Àsà àti ìse Yorùbá C. L. Adéoyé.pdf']

In [ ]:
import os
from mistralai import Mistral

api_key = "GQgpzUGD5yR8EzjzVSNYKLtvBhJnkbxQ"
client = Mistral(api_key=api_key)



In [7]:
import os
import base64
import pandas as pd
from mistralai import Mistral
from docx import Document



# 📂 Path to Books folder
books_folder = path_book

# 🔹 Helper: OCR with Mistral for PDFs
def ocr_pdf(path):
    # Upload the PDF file
    with open(path, "rb") as f:
        uploaded_file = client.files.upload(
            file={"file_name": os.path.basename(path), "content": f.read()},
            purpose="ocr"
        )
    # Get a signed URL for the uploaded file
    signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)
    # Process the PDF with Mistral OCR
    ocr_response = client.ocr.process(
        document={"type": "document_url", "document_url": signed_url.url},
        model="mistral-ocr-latest",
        include_image_base64=False
    )
    # Combine all extracted text from all pages
    if hasattr(ocr_response, "pages"):
        return "\n".join([page.markdown for page in ocr_response.pages])
    return ""

# 🔹 Helper: Extract text from DOCX
def read_docx(path):
    doc = Document(path)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])

# 🔹 Split text into chunks of 1000 characters
def split_chunks(text, size=1000):
    return [text[i:i+size] for i in range(0, len(text), size)]

# 🔹 Collect rows
rows = []

for file in os.listdir(books_folder):
    path = os.path.join(books_folder, file)
    text = ""

    if file.lower().endswith(".pdf"):
        print(f"📖 OCR processing PDF: {file}")
        text = ocr_pdf(path)

    elif file.lower().endswith(".docx"):
        print(f"📖 Extracting DOCX: {file}")
        text = read_docx(path)

    else:
        print(f"⚠️ Skipping unsupported file: {file}")
        continue

    if not text.strip():
        print(f"⚠️ No text extracted from {file}")
        continue

    # Split and save chunks
    chunks = split_chunks(text, 1000)
    for c in chunks:
        rows.append({"domain": "books", "source": file, "content": c})

# 🔹 Save to CSV
df = pd.DataFrame(rows)
df.to_csv("yoruba_books_mistral.csv", index=False, encoding="utf-8")

print(f"✅ Saved {len(df)} rows from {len(rows)} chunks into yoruba_books_mistral.csv")



📖 OCR processing PDF: 700267500-Agbeyewo-Ilo-ede-ati-eto-iselu-ninu-Ere-Onitan-Aye-Ye-Won-Tan-lati-owo-Akinwunmi-Isola-ati-Iselu-Onireke-Oge-lati-owo-Lere-Adeyemi.pdf
📖 Extracting DOCX: Asa and orisa.docx
📖 Extracting DOCX: asa.docx
⚠️ Skipping unsupported file: extract.ipynb
📖 OCR processing PDF: YOR213 - Use of Yoruba.pdf
📖 OCR processing PDF: YOR215 - Introduction to Yoruba Oral Literature.pdf
⚠️ Skipping unsupported file: ~WRL0740.tmp
📖 Extracting DOCX: Àsà àti ìse Yorùbá C. L. Adéoyé.docx
📖 OCR processing PDF: Àsà àti ìse Yorùbá C. L. Adéoyé.pdf
✅ Saved 2269 rows from 2269 chunks into yoruba_books_mistral.csv


In [58]:
import os
import pandas as pd
from pathlib import Path
from pdf2image import convert_from_path
import pytesseract
import docx

# Function to OCR PDF using Tesseract
def read_pdf_ocr(file_path):
    text = ""
    try:
        images = convert_from_path(file_path, dpi=300)  # Convert each page to image
        for img in images:
            text += pytesseract.image_to_string(img, lang="yor") + "\n"
    except Exception as e:
        print(f"⚠️ Could not OCR {file_path}: {e}")
    return text.strip()

# Function to read DOCX directly (if text-based) else fallback to OCR
def read_docx(file_path):
    text = ""
    try:
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    except Exception as e:
        print(f"⚠️ Could not read {file_path}: {e}")
    return text.strip()

# Split into 1000-char chunks
def split_chunks(text, size=1000):
    return [text[i:i+size] for i in range(0, len(text), size)]

# Process Books folder
books_folder = path_book
rows = []

for file in os.listdir(books_folder):
    path = os.path.join(books_folder, file)
    content = ""
    
    if file.lower().endswith(".pdf"):
        content = read_pdf_ocr(path)
    elif file.lower().endswith(".docx"):
        content = read_docx(path)  # text-based
        # Optional: convert to OCR if mostly scanned images
    else:
        continue
    
    if content:
        chunks = split_chunks(content, 1000)
        for chunk in chunks:
            rows.append({
                "domain": "books",
                "source": file,
                "text": chunk
            })

# Save CSV
df_books = pd.DataFrame(rows)
df_books.to_csv("yoruba_books_ocr.csv", index=False, encoding="utf-8")
print(f"✅ OCR completed: {len(df_books)} rows extracted")


⚠️ Could not OCR C:\Users\Lightified\Desktop\RAG\books\700267500-Agbeyewo-Ilo-ede-ati-eto-iselu-ninu-Ere-Onitan-Aye-Ye-Won-Tan-lati-owo-Akinwunmi-Isola-ati-Iselu-Onireke-Oge-lati-owo-Lere-Adeyemi.pdf: Unable to get page count. Is poppler installed and in PATH?
⚠️ Could not OCR C:\Users\Lightified\Desktop\RAG\books\YOR213 - Use of Yoruba.pdf: Unable to get page count. Is poppler installed and in PATH?
⚠️ Could not OCR C:\Users\Lightified\Desktop\RAG\books\YOR215 - Introduction to Yoruba Oral Literature.pdf: Unable to get page count. Is poppler installed and in PATH?
⚠️ Could not read C:\Users\Lightified\Desktop\RAG\books\~$a and orisa.docx: Package not found at 'C:\Users\Lightified\Desktop\RAG\books\~$a and orisa.docx'
⚠️ Could not read C:\Users\Lightified\Desktop\RAG\books\~$asa.docx: Package not found at 'C:\Users\Lightified\Desktop\RAG\books\~$asa.docx'
⚠️ Could not read C:\Users\Lightified\Desktop\RAG\books\~$à àti ìse Yorùbá C. L. Adéoyé.docx: Package not found at 'C:\Users\Lightif

In [59]:
df_books.source.unique()

array(['Asa and orisa.docx', 'asa.docx',
       'Àsà àti ìse Yorùbá C. L. Adéoyé.docx'], dtype=object)

In [62]:
df_books

,domain,source,text
0,books,Asa and orisa.docx,ỆKỘ KINÍ\nỆKỘ-ILE TABI KĪKÍ NI\nAwon Yoruba ma...
1,books,Asa and orisa.docx,", Ayinla""\nomo Akin, o k'aaro, o o jiire bi?"" ..."
2,books,asa.docx,Ìtumo Yorùbá\nYoruba ni: Edè gbogbo ni a gbọ́ ...
3,books,asa.docx,ko yii. A sì fi yé wa pé\ngbogho awon tí ó gba...
4,books,asa.docx,kédájó ni Ọlorun yii. Afúnnimásèrègún ni\nati ...
...,...,...,...
1040,books,Àsà àti ìse Yorùbá C. L. Adéoyé.docx,"hàn pé àw9n ljęşà ń şe.\nlŞ,E QWQ ÀTl ÒWÒ\nK ..."
1041,books,Àsà àti ìse Yorùbá C. L. Adéoyé.docx,"ú) Çłwğ qmqdé kò tó pępę, ti àgbalagbà kò wq :..."
1042,books,Àsà àti ìse Yorùbá C. L. Adéoyé.docx,"ğdędę baálé ?.\n\nàti ,märùn-ún tí 'awęn obìn..."
1043,books,Àsà àti ìse Yorùbá C. L. Adéoyé.docx,ni å ń pè ní ‘lwàręfà’ ?\n(b) Brtwo ni àwqn\ty...
